In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt

# vectorization
from sklearn.preprocessing import LabelEncoder

# metrics
from sklearn.metrics import accuracy_score

# misc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import make_pipeline

# models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [2]:
# load train and test data

train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

## 1. Structure of the Data

Size of **training examples**: (891,12) <br/>
Size of **test data**: (418,11)

In [3]:
print(f'Train dataset size: {train.shape}')
train.head()

Train dataset size: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
print(f'Test dataset size: {test.shape}')
test.head()

Test dataset size: (418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## 2. Missing Values

In [5]:
def missing_values(df, columns):
    """ Finds number of rows where specified columns are missing values.
    
    Args:
        df:
            The dataframe to be analyzed.
        columns:
            The list of columns of the dataframe.
    
    Returns:
         A dictionary with the columns and the number of values they are missing.
    """
    missing = {}
    
    for column in columns:
        total = df[column].value_counts().sum()
        missing[column] = df.shape[0] - total
    
    return missing

In [ ]:
missing = missing_values(train, train.columns)
print(f'Total values that each column should have: {train.shape[0]}')
print(f'Total values that each column is missing:')
missing

**Observations:**
* The **PassengerId** and **Name** columns have unique values for each row in the trainning examples. Thus, they won't help in classification. <br/>
* We can create an new feature, called **Title** which will be the title of each person.
* The **Age** column is missing a lot of values. We'll fill them using the mean of the ages with respect to the persons title . <br/>
* The **Cabin** column has a lot of missing values. We'll only keep the first letter of the cabin and create a new cabin class, namely 'n' for these rows that have missing values in this column. <br/>
* The **Embarked** column has 2 missing values. So, we'll remove these 2 rows.

**Note:**
* Creating a new cabin class for those rows that didn't have a value in this column didn't work. Maybe, I can take the mean with respect to the ticket class. I'll assume that the higher the class is, the better the cabin class is.

## 3. Number of Unique Values

In [6]:
print(f'Ticket column has {len(train.Ticket.unique())} different values.')

Ticket column has 681 different values.


In [7]:
print(f'Fare column has {len(train.Fare.unique())} different values.')

Fare column has 248 different values.


**Observations:**
* The **Ticket** column has a lot of different values. Specifically, 681/891 different values. For now, we'll drop this column.
* The **Fare** column has a quite many different values. We'll create ranges.

**Notes:**
* Can I group together the different values in ticket column?

## 4. Preprocess

In [8]:
# create a column with the titles

train['Title'] = train['Name'].str.extract(', (\w+)\.', expand=False)

In [9]:
# fill missing data in "Age" column using mean with respect to the "Title" column

train['Age'] = train['Age'].fillna(train.groupby('Title')['Age'].transform('median'))

In [10]:
# keep the first letter from the cabin
train['Cabin'] = train['Cabin'].str[:1]

# fill missing data in "Cabin" column using 'n' (we'll assume that n is a cabin class)
train['Cabin'] = train['Cabin'].fillna(value='n')

In [11]:
# create bins for "Fare" and "Age" (discretise)

train['Fare'] = pd.cut(train['Fare'], bins=5)
train['Age'] = pd.cut(train['Age'], bins=10)

In [12]:
# remove rows

train = train.dropna(subset=['Embarked'])

In [13]:
# remove columns

train = train.drop(['PassengerId', 'Name' ,'Ticket'], axis=1)

In [14]:
print('Training examples after preprocessing:')
train.head()

Training examples after preprocessing:


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,0,3,male,"(16.336, 24.294]",1,0,"(-0.512, 102.466]",n,S,Mr
1,1,1,female,"(32.252, 40.21]",1,0,"(-0.512, 102.466]",C,C,Mrs
2,1,3,female,"(24.294, 32.252]",0,0,"(-0.512, 102.466]",n,S,Miss
3,1,1,female,"(32.252, 40.21]",1,0,"(-0.512, 102.466]",C,S,Mrs
4,0,3,male,"(32.252, 40.21]",0,0,"(-0.512, 102.466]",n,S,Mr


## 5. Vectorize

We'll try categorical encoding vectors.

In [16]:
train_enc_cat = train.apply(LabelEncoder().fit_transform)

In [17]:
print(f'Categorical encoding vector size: {train_enc_cat.shape}')
train_enc_cat.head()

Categorical encoding vector size: (889, 10)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,0,2,1,2,1,0,0,8,2,11
1,1,0,0,4,1,0,0,2,0,12
2,1,2,0,3,0,0,0,8,2,8
3,1,0,0,4,1,0,0,2,2,12
4,0,2,1,4,0,0,0,8,2,11


## 6. Fitting Models and Score

In [18]:
y_train = train_enc_cat.Survived
X_train = train_enc_cat.drop(['Survived'], axis=1)

In [19]:
print(f'Train data size: {X_train.shape}')
X_train.head()

Train data size: (889, 9)


,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,2,1,2,1,0,0,8,2,11
1,0,0,4,1,0,0,2,0,12
2,2,0,3,0,0,0,8,2,8
3,0,0,4,1,0,0,2,2,12
4,2,1,4,0,0,0,8,2,11


In [20]:
print(f'Train labels size: {y_train.shape}')
y_train.head()

Train labels size: (889,)


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

### 6.1 Logistic Regression

In [22]:
lr = LogisticRegression()

# cross validation
y_pred = cross_val_predict(lr, X_train, y_train, cv=10)

# accuracy
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy of Logistic Regression: {accuracy}')

Accuracy of Logistic Regression: 0.7851518560179978


### 6.2 Decision Tree (Archived)

In [ ]:
dt = DecisionTreeClassifier()

# cross validation
y_pred = cross_val_predict(dt, X_train, y_train, cv=10)

# accuracy
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy for Decision Tree: {accuracy}')

### 6.3 Random Forest (Archived)

In [ ]:
# 100 decision tree classifiers
# feture selection with gini
# decision trees have max depth
rf = RandomForestClassifier()

# cross validation
y_pred = cross_val_predict(rf, X_train, y_train, cv=10)

# accuracy
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy for Random Forest: {accuracy}')

### 6.4 Support Vector Machines

In [ ]:
# values will be scaled using equation (val-mean)/std
svc = make_pipeline(StandardScaler(), LinearSVC())

# cross validation
y_pred=cross_val_predict(svc, X_train, y_train, cv=10)

# accuracy
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy for Linear SVC: {accuracy}')

## 7. Predict on Test Data

We'll perform the same transformations to the test data.

### 7.1 Preprocessing

In [23]:
# create a column with the titles
test['Title'] = test['Name'].str.extract(', (\w+)\.', expand=False)

In [24]:
# insert missing data in "Age" column using mean with respect to the "Title" column
test['Age'] = test['Age'].fillna(test.groupby('Title')['Age'].transform('median'))

In [25]:
# keep the first letter from the cabin
train['Cabin'] = train['Cabin'].str[:1]

# fill missing data in "Cabin" column using 'n' (we'll assume that n is a cabin class)
train['Cabin'] = train['Cabin'].fillna(value='n')

In [26]:
# create bins for "Fare" and "Age" (discretise)
test['Fare'] = pd.cut(test['Fare'], bins=5)
test['Age'] = pd.cut(test['Age'], bins=10)

In [27]:
# test data should have the same format (columns) as training examples
columns = X_train.columns

### 7.2 Vectorize

In [28]:
test_enc_cat = test[columns].apply(LabelEncoder().fit_transform)

### 7.3 Predict with Best Model

Best model up to now is logistic regression classifier, with training accuracy 77%.

In [29]:
X_test = test_enc_cat

In [32]:
best_model = lr

# train model
best_model.fit(X_train, y_train)

# predict on test data
y_pred = best_model.predict(X_test)

## 8 Submission

In [33]:
submission = pd.DataFrame()
submission['PassengerId'] = test['PassengerId']
submission['Survived'] = y_pred

In [34]:
print(f'Submission size: {submission.shape}')
submission.head()

Submission size: (418, 2)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [36]:
submission.to_csv('/kaggle/working/linear-regression.csv', index=False)